In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random

# 定义数据集路径
data_folder = 'qmnist/all'
x_data = []
y_data = []

# 加载图像和标签
for label in range(10):
    label_folder = os.path.join(data_folder, str(label))
    images = []

    for img_name in os.listdir(label_folder):
        img_path = os.path.join(label_folder, img_name)
        
        # 加载并处理图像
        image = Image.open(img_path).convert('L')  # 转换为灰度图
        image = image.resize((28, 28))  # 调整大小
        image_array = np.array(image)
        images.append(image_array)
    
    x_data.extend(images)
    y_data.extend([label] * len(images))

# 转换为NumPy数组
x_data = np.array(x_data)
y_data = np.array(y_data)

# 直接使用所有样本
x_balanced = x_data
y_balanced = y_data

# 按比例拆分数据集（例如，80% 训练集，20% 测试集）
x_train, x_test, y_train, y_test = train_test_split(x_balanced, y_balanced, test_size=0.2, random_state=42)

# 确保数据集的形状符合TensorFlow的要求
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') 
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32')

# CREATE MORE IMAGES VIA DATA AUGMENTATION
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)
# 适配数据生成器
datagen.fit(x_train)

# 打印每种图片的数量
unique, counts = np.unique(y_balanced, return_counts=True)
print("每种图片数量:")
for label, count in zip(unique, counts):
    print(f'类别 {label}: {count} 张')


In [ ]:
import numpy as np
from keras.datasets import mnist
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input,Conv2D, Dense, MaxPooling2D, Softmax, Activation, BatchNormalization, Flatten, Dropout, DepthwiseConv2D
from tensorflow.keras.layers import MaxPool2D, AvgPool2D, AveragePooling2D, GlobalAveragePooling2D,ZeroPadding2D,Input,Embedding,PReLU,Reshape
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
import keras.backend as K
import tensorflow as tf
import time
# 
# from os import environ
# environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
# (x_train,y_train), (x_test,y_test) = mnist.load_data() 


In [ ]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)/255
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)/255

y_train = to_categorical(y_train,num_classes=10) 
y_test = to_categorical(y_test,num_classes=10)

In [ ]:
#mnist_dw

def init_model(dim0):
    model = Sequential()

    # 第一个卷积层
    model.add(Conv2D(dim0, (3, 3), padding='same', strides=(2, 2),
                     input_shape=(28, 28, 1), name='ftr0a'))
    model.add(BatchNormalization(name="bn0"))
    model.add(Activation('relu', name="relu0"))
    
    # 第一个深度可分离卷积层
    model.add(DepthwiseConv2D((3, 3), padding='same', name='ftr0b'))
    model.add(BatchNormalization())
    model.add(Activation('relu', name="relu00"))

    # 第二个卷积层
    model.add(Conv2D(dim0 * 4, (3, 3), padding='same', strides=(2, 2), name='ftr1a'))
    model.add(BatchNormalization(name="bn1"))
    model.add(Activation('relu', name="relu1"))
    
    # 第二个深度可分离卷积层
    model.add(DepthwiseConv2D((3, 3), padding='same', depth_multiplier=2, name='ftr1b'))
    model.add(BatchNormalization())
    model.add(Activation('relu', name="relu11"))

    model.add(Dropout(0.3))  # 添加 Dropout 层

    # 全局平均池化层
    model.add(GlobalAveragePooling2D(name='GAP'))
    model.add(Dropout(0.3))  # 添加 Dropout 层
    
    # 全连接层
    model.add(Dense(10, name="fc1"))
    model.add(Activation('softmax', name="sm"))

    return model

# 设置参数并初始化模型
DIM0 = 4
model = init_model(DIM0)
model.summary()

In [ ]:
# EPOCHS = 20
# model.compile(optimizer='adam', loss = "categorical_crossentropy", metrics = ["categorical_accuracy"]) 
# H = model.fit(x_train, y_train, batch_size=64, epochs= EPOCHS,  verbose= 1, validation_data = (x_test, y_test), shuffle=True) 

In [ ]:
# 训练模型
EPOCHS = 20

# 编译模型
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["categorical_accuracy"])

# 创建数据增强生成器
train_generator = datagen.flow(x_train, y_train, batch_size=128)



# 使用数据生成器进行训练
model.fit(train_generator,
          epochs=EPOCHS,
          verbose=1,
          # validation_data=(x_test, y_test),  # 保持验证数据不变
           validation_data=datagen.flow(x_test, y_test),
          shuffle=True)


In [ ]:
h5_file = "mnist.h5"
model.save(h5_file)
model.export("mnist")

In [ ]:
data = x_test[1]
for y in range(28):
    for x in range(28):
        print("%3d,"%(int(data[y,x,0]*255)), end="")
    print("")

In [ ]:
result = model.predict(x_test[1:2])[0]
result

In [ ]:
# 调用转换脚本
!python tools/h5_to_tflite.py mnist mnist.tflite 1 qmnist/all 0to1
!python tools/tflite2tmdl.py mnist.tflite mnist.tmdl int8 1 28,28,1 10 1